In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [1]:
import triton_python_backend_utils as pb_utils

In [33]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [34]:
import pandas as pd
import numpy as np
import json

In [35]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [36]:
data_infer = pd.read_csv("data_infer.csv").astype('str')

In [119]:
input_data = data_infer.values

In [120]:
data_infer.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?
0,1005,0,2013,6,5,22:03,$-180.00,Swipe Transaction,7834055923142137930,Orlando,FL,32808.0,3395,nan
1,134,5,2015,2,16,18:34,$99.10,Chip Transaction,-1548923525906069124,Tyler,TX,75706.0,4900,nan
2,1270,0,2009,11,27,18:44,$82.42,Swipe Transaction,-7329090674102508598,Saint Charles,IL,60174.0,5812,nan
3,109,0,2010,11,3,16:21,$0.45,Swipe Transaction,-6571010470072147219,Dyer,IN,46311.0,5499,nan
4,1853,2,2006,3,27,12:25,$11.41,Swipe Transaction,6135208568923449408,East Peoria,IL,61611.0,9402,nan


In [121]:
input_data

array([['1005', '0', '2013', ..., '32808.0', '3395', 'nan'],
       ['134', '5', '2015', ..., '75706.0', '4900', 'nan'],
       ['1270', '0', '2009', ..., '60174.0', '5812', 'nan'],
       ...,
       ['1436', '0', '2017', ..., '15083.0', '5411', 'nan'],
       ['1227', '3', '2011', ..., '29525.0', '5311', 'nan'],
       ['1309', '1', '2013', ..., '45869.0', '5912', 'nan']], dtype=object)

In [122]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [123]:
model_name = "ensemble_preprocess_fil_postprocess"

In [124]:
inputs = [prepare_tensor('raw_data', input_data)]

In [125]:
input_data.shape

(100, 14)

In [118]:
response = client.infer(model_name,
                        inputs)

InferenceServerException: in ensemble 'ensemble_preprocess_fil_postprocess', unexpected shape for input 'CLASS_IDX' for model 'postprocessing'. Expected [-1,1], got [3]

In [98]:
output_data = response.as_numpy("postprocessed_predictions")

In [99]:
np.set_printoptions(suppress=True)

In [100]:
output_data

array([[ 1005.  ,     0.  ,  2013.  ,     6.  ,     5.  ,  -180.  ,
            2.  , 71455.  ,  8544.  ,    58.  ,  7326.  ,    21.  ,
            0.  ,    22.  ,     3.  ],
       [  134.  ,     5.  ,  2015.  ,     2.  ,    16.  ,    99.1 ,
            0.  ,  2578.  , 11757.  ,   189.  , 19836.  ,    45.  ,
            0.  ,    18.  ,    34.  ],
       [ 1270.  ,     0.  ,  2009.  ,    11.  ,    27.  ,    82.42,
            2.  , 29794.  , 10074.  ,    78.  , 15541.  ,    69.  ,
            0.  ,    18.  ,    44.  ]], dtype=float32)

In [90]:
output_data.shape

(3, 15)

In [11]:
data = pd.DataFrame(['nan'], columns=['Errors?']).astype(str)

In [13]:
data['Errors?'] != 'nan'

0    False
Name: Errors?, dtype: bool

In [86]:
print(output_data)

[45.23]


In [105]:
CLASS_LABELS = ['NOT FRAUD', 'FRAUD']

In [106]:
class_indices = np.array([1.0, 0.0, 1.1, 1.1])
map_func = lambda idx : CLASS_LABELS[int(idx)]
class_labels = map_func(class_indices)

TypeError: only size-1 arrays can be converted to Python scalars